In [ ]:
!pip install langchain
!pip install python-dotenv
!pip install langchain-community
!pip install huggingface_hub
!pip install langchain-groq
!pip install chromadb
!pip install sentence_transformers
!pip install langchain_huggingface
!pip install langchain_chroma
!pip install bs4
!pip install langchain-text-splitters

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

### Conversation Q&A Chatbot
In many Q&A applications we want to allow the user to have a back-and-forth conversation.

In this guide we focus on adding logic for incorporating historical messages.

We will cover two approaches:

- Chains, in which we always execute a retrieval step;
- Agents, in which we give an LLM discretion over whether and how to execute a retrieval step (or multiple steps).

In [ ]:
import os
from google.colab import userdata
from langchain_groq import ChatGroq

groq_api_key = userdata.get('GROQ_API_KEY')

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

llm



ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f1c1758a050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f1c17ab9c90>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
import bs4
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs=loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f1ad8e8c0d0>, search_kwargs={})

### With Chain

In [65]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,prompt) # create stuff document will create a combined single doc from multiple docs
rag_chain=create_retrieval_chain(retriever,question_answer_chain) # question answer chain contains llm and prompt.

##  response contain the output structure of create_stuff_document_chain

In [ ]:
response=rag_chain.invoke({"input":"What is Self-Reflection"})
response

{'input': 'What is Self-Reflection',
 'context': [Document(id='a36d16e1-0729-4c87-b8ad-e90b8d96b24d', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(id='7841fdec-bc36-40fe-8a49-88029ac95c90', metadata={'source': 'https://lilianweng.g

In [66]:
response['answer']

'Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

### Adding Chat History

In [67]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

# this prompt template take user question/ conversation history/ prompt to create standalone question
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

## create_history_aware_retriever(llm,retriever,contextualize_q_prompt)  This function takes this three parameter and return retriever like object

In [68]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f1ad8e8c0d0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChu

In [69]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),  # here chat history is again provided for case where history is needed to answer.
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt) # return combine documents
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain) # question answer chain contains llm and prompt.

In [ ]:
# from langchain_core.messages import AIMessage,HumanMessage
# chat_history=[]
# question="What is Self-Reflection"
# response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

# chat_history.extend(    # this is manual way you can use RunnableWithMessageHistory to automate it done below
#     [
#         HumanMessage(content=question),
#         AIMessage(content=response1["answer"])
#     ]
# )

# question2="Tell me more about it?"
# response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
# print(response2['answer'])

Self-Reflection is a process that enables autonomous agents to improve by refining past actions and correcting mistakes, allowing them to iteratively learn and adapt.


In [ ]:
chat_history

[HumanMessage(content='What is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.', additional_kwargs={}, response_metadata={})]

## Automating memory storage

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",  # Key for user input in the request
    history_messages_key="chat_history", # Key for chat history since we have mentioned here so no need to mention it while invoking. 
    output_messages_key="answer",  # Key for storing AI responses If you don't provide output_messages_key" in RunnableWithMessageHistory, then the AI's response will not be explicitly saved in history.
)

config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse

- Here i have not provided output_messages_key="answer". see below
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages" #we need to specify the correct key to use to save the chat history.
)
- but it is able to save the context but why not in above code

## Answer
Your chain might be returning a list of messages, such as:
[AIMessage(content="Hello Krish, how can I help?")] in this case
Since RunnableWithMessageHistory automatically saves AIMessage instances when they are returned in a list format, it works even without output_messages_key.

- but in above case output is coming as a dict.
- Since you didn't specify output_messages_key="answer", LangChain does not know where to extract the AI's response from the dictionary and thus does not store it.

In [89]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. This is often done to help agents, such as LLM-powered autonomous agents, plan and execute tasks more effectively.'

In [90]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. This is often done to help agents, such as LLM-powered autonomous agents, plan and execute tasks more effectively.', additional_kwargs={}, response_metadata={})])}

In [91]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)

{'input': 'What are common ways of doing it?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. This is often done to help agents, such as LLM-powered autonomous agents, plan and execute tasks more effectively.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='7c702102-7f64-4dd6-97b7-ca88df4513bf', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majori

In [92]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. This is often done to help agents, such as LLM-powered autonomous agents, plan and execute tasks more effectively.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Common ways of doing task decomposition include using LLMs with simple prompting, task-specific instructions, or human inputs.', additional_kwargs={}, response_metadata={})])}